# Running ROUGE Evaluation for Three Models

This notebook will feed each prompt from the dataset into three models (e.g., base / fine-tuned), compute the ROUGE score for each sample using the project's `rouge_calculator.py`, and save the aggregated statistics.

Note: Please modify the sections marked as **TODO** according to your environment (model checkpoint paths, cache directory, whether to use CUDA, etc.).

## Dependencies (if needed)

If you are running this in Colab or a new environment, please install the required dependencies first.

Below are the recommended installation commands to run in Colab (including commonly used acceleration and evaluation libraries):


In [ ]:
!pip install -U pip setuptools wheel
!pip install -U transformers datasets peft bitsandbytes rouge-score tqdm accelerate einops rouge-score finnhub-python

from google.colab import drive
drive.mount('/content/drive')

print('Installation complete')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.5 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 151.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 157.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 127.5 MB/s  0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24987 sha256=e6713d0ab2375cd7f68239a62ddcf78f555da94e28959ecf1e1ca499a1ce9fe7
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
  Attempting unin

In [ ]:
import os
import sys
import json
from types import SimpleNamespace
from pprint import pprint
from google.colab import userdata
PROJECT_ROOT = '/content/juankim834'  # TODO: Change to your Path
!unzip /content/drive/MyDrive/juankim834.zip -d /content/juankim834
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')
os.environ["WANDB_API_KEY"] = userdata.get('WANDB_FOR_FINGPT')
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)




Archive:  /content/drive/MyDrive/juankim834.zip
   creating: /content/juankim834/finetuned_models/
   creating: /content/juankim834/finetuned_models/deepseek-ai/
   creating: /content/juankim834/finetuned_models/deepseek-ai/DeepSeek-R1-Distill-Llama-8B_202602210707/
  inflating: /content/juankim834/finetuned_models/deepseek-ai/DeepSeek-R1-Distill-Llama-8B_202602210707/adapter_config.json  
  inflating: /content/juankim834/finetuned_models/deepseek-ai/DeepSeek-R1-Distill-Llama-8B_202602210707/adapter_model.safetensors  
  inflating: /content/juankim834/finetuned_models/deepseek-ai/DeepSeek-R1-Distill-Llama-8B_202602210707/chat_template.jinja  
   creating: /content/juankim834/finetuned_models/deepseek-ai/DeepSeek-R1-Distill-Llama-8B_202602210707/checkpoint-39/
  inflating: /content/juankim834/finetuned_models/deepseek-ai/DeepSeek-R1-Distill-Llama-8B_202602210707/checkpoint-39/adapter_config.json  
  inflating: /content/juankim834/finetuned_models/deepseek-ai/DeepSeek-R1-Distill-Llama-8B

In [ ]:
import os
os.chdir(PROJECT_ROOT)
!ls

finetuned_models  rouge_calculator.py	       utils
llm_test	  run_three_models_eval.ipynb


In [ ]:
from llm_test import model_runner

## TODO: Configure the paths for the three models (replace the placeholders below with the actual paths in your Colab/Drive environment)

In [ ]:
# TODO: Replace your path and 
models = [
    # {
    #     'label': 'model_llama3-base',
    #     'model_name': 'meta-llama/Llama-3.1-8B-Instruct',  # base model name (HuggingFace repo name)
    #     'fine_tuned_path': None,
    #     'cache_dir': '/content/juankim834/pretrained_models',  # TODO: Change to your Path of Cache
    #     'use_cuda': True,
    # },
    {
        'label': 'model_finetuned_qwen',
        'model_name': 'Qwen/Qwen2.5-7B-Instruct',
        'fine_tuned_path': '/content/juankim834/finetuned_models/Qwen/Qwen2.5-7B-Instruct_202602201649/checkpoint-39', # TODO: Change to your Path of fine-tuned Cache
        'cache_dir': '/content/juankim834/pretrained_models',
        'use_cuda': True,
    },
    {
        'label': 'model_finetuned_llama3',
        'model_name': 'meta-llama/Meta-Llama-3.1-8B-Instruct',
        'fine_tuned_path': '/content/juankim834/finetuned_models/meta-llama/Meta-Llama-3-8B-Instruct_202602201606/checkpoint-50', # TODO: Change to your Path of fine-tuned Cache
        'cache_dir': '/content/juankim834/pretrained_models',
        'use_cuda': True,
    },
    {
        'label': 'model_finetuned_deepseek',
        'model_name': 'deepseek-ai/DeepSeek-R1-Distill-Llama-8B',
        'fine_tuned_path': '/content/juankim834/finetuned_models/deepseek-ai/DeepSeek-R1-Distill-Llama-8B_202602210707', # TODO: Change to your Path of fine-tuned Cache
        'cache_dir': '/content/juankim834/pretrained_models',
        'use_cuda': True,
    },
]

# Evaluation hyperparameters
common_args = {
    'dataset': 'dow30-202305-202405',
    'from_remote': True,
    'split': 'test',
    'start_date': None,
    'end_date': None,
    'symbol': None,
    'max_samples': 0,
    'max_length': 2048,
    'temperature': 0.7,
    'output_dir': '/content/drive/MyDrive/rouge_results',
    'device': 'cuda:0',
    'batch_size': 12
}

print('Configuring model complete, please check your model configuration again')
# Get tokens
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')
os.environ["WANDB_API_KEY"] = userdata.get('WANDB_FOR_FINGPT')
from huggingface_hub import login
login(token=os.environ["HF_TOKEN"]) # Implicitly login fails sometimes, use explicit way to login again

模型配置示例就绪。请检查并修改 TODO 路径。


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [ ]:
# # Smoke test
# # To confirm model generation ability
# import gc
# import torch
# import rouge_calculator
# import torch
# # Load one sample from dataset. 
# ds_list = rouge_calculator.load_dataset(common_args['dataset'], from_remote=common_args['from_remote'])
# ds = ds_list[0].get(common_args['split'])
# # Filtering Samples
# try:
#     filtered = ds.filter(lambda x: common_args['start_date'] <= x['period'] <= common_args['end_date'] and x['symbol'] == common_args['symbol'])
# except Exception:
#     filtered = ds
# if len(filtered) == 0:
#     print('No filtered samples found; using first sample from split')
#     item = ds[0]
# else:
#     item = filtered[0]
# prompt = item['prompt']
# ground_truth = item['answer']
# print('=== Ground truth preview ===')
# print(ground_truth[:500])

# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# print('Using device:', device)

# for m in models:
#     print('--- Testing model:', m['label'], '---')
#     try:
#         runner = model_runner(m['model_name'], cache_dir=m['cache_dir'], device=device, fine_tuned_path=m['fine_tuned_path'])
#         gen = runner.generate_from_one_prompt(prompt, max_length=512, temperature=0.7)
#         print('Generated (truncated 100 chars):')
#         print(gen[:100])
#     except Exception as e:
#         print('Error while testing', m['label'], e)
#     gc.collect()
#     if torch.cuda.is_available():
#         torch.cuda.empty_cache()
# gc.collect()
# if torch.cuda.is_available():
#     torch.cuda.empty_cache()


In [ ]:
import gc
import torch
import rouge_calculator
from types import SimpleNamespace
results = {}
for m in models:
    print('Running eval for:', m['label'])
    args_ns = SimpleNamespace(**common_args)
    args_ns.model_name = m['model_name']
    args_ns.fine_tuned_path = m['fine_tuned_path']
    args_ns.cache_dir = m['cache_dir']
    args_ns.use_cuda = m['use_cuda']
    args_ns.output_dir = os.path.join(common_args['output_dir'], m['label'])
    os.makedirs(args_ns.output_dir, exist_ok=True)

    per_path, summary_path = rouge_calculator.evaluate(args_ns)
    print('Finished:', m['label'])
    results[m['label']] = { 'per_sample': per_path, 'summary': summary_path }
    gc.collect() # Force to unload memory of CUDA and RAM
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

with open(os.path.join(common_args['output_dir'], 'runs_index.json'), 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print('All runs finished. Index saved to', os.path.join(common_args['output_dir'], 'runs_index.json'))

Running eval for: model_finetuned_qwen


README.md:   0%|          | 0.00/581 [00:00<?, ?B/s]

data/train-00000-of-00001-7c4c80aa07272d(…):   0%|          | 0.00/3.57M [00:00<?, ?B/s]

(…)-00000-of-00001-28531804b005ddc6.parquet:   0%|          | 0.00/925k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1230 [00:00<?, ? examples/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Generating and scoring: 100%|██████████| 25/25 [15:07<00:00, 36.30s/it]


Saved per-sample results to: /content/drive/MyDrive/rouge_results/model_finetuned_deepseek/per_sample_rouge.json
Saved summary stats to: /content/drive/MyDrive/rouge_results/model_finetuned_deepseek/summary_rouge.json
Memory usage after cleaning: 9.12 MB
Finished: model_finetuned_deepseek
All runs finished. Index saved to /content/drive/MyDrive/rouge_results/runs_index.json


In [ ]:
# This is for saving calculation units. 
from google.colab import runtime
runtime.unassign()

## Quick Start for Colab (CLI Examples)
Below is an example of the shell commands used to execute the complete evaluation pipeline in a Google Colab environment:

In [ ]:
# Use to run in the colab environment with only few commands, but running this notebook is also practical
# from google.colab import drive
# drive.mount('/content/drive')

# %cd /content/drive/MyDrive/juankim834

# !pip install -U transformers datasets peft bitsandbytes rouge-score tqdm accelerate

# !python rouge_calculator.py --dataset dow30-202305-202405 --from_remote \
#    --split test --start_date 2024-01-07 --end_date 2024-01-14 --symbol GS \
#    --model_name 'Qwen/Qwen2.5-7B-Instruct' --fine_tuned_path '/content/drive/MyDrive/finetuned_models/Qwen/checkpoint-39' \
#    --use_cuda --output_dir '/content/drive/MyDrive/rouge_results/model_finetuned_qwen'

In [ ]:
# 
index_path = '/content/drive/MyDrive/rouge_results/runs_index.json'
if os.path.exists(index_path):
    with open(index_path, 'r', encoding='utf-8') as f:
        runs = json.load(f)
    pprint(runs)
else:
    print('Index file not found at', index_path)